# Conexiones módulo PWM SG3525
[Datasheet SG3525A](/Datasheets/SG3525A-D.PDF)

## Frecuencia
Cálculo de $C_T$ y $R_T$ para fijar $f_{sw}$

$$f_{sw} = \frac{1}{C_T \cdot (0.7R_T + 3R_D)}$$

Como no necesitamos deadtime $\Rightarrow R_D = 0$

Queremos $f_{sw} = 75 kHz$

In [1]:
from utils import C_values, R_values, to_unit, parallel

fsw = 75E3
Rd = 0

tol = 0.005
CT = []
RT = []

for Ct in C_values:
    if 1E-9 < Ct < 200E-9:
        for Rt in R_values:
            if 2E3 < Rt < 150E3: 
                f = 0.5/(Ct*(0.7*Rt + Rd*3))
                if fsw*(1-tol) < f < fsw*(1+tol):
                    CT.append(Ct)
                    RT.append(Rt)
        
print(f'Se encontraron {len(CT)} opciones')
for i in range(len(CT)):
    print('CT = ' + to_unit(CT[i]) + 'F con RT = ' + to_unit(RT[i]) + 'Ohm, resultando en fsw = ' + to_unit(0.5/(CT[i]*(0.7*RT[i] + Rd*3))) + 'Hz')

if len(CT) == 0:
    print('Buscando con resistencias en paralelo\n')

    Rpar = []
    for Ct in C_values:
        if 1E-9 < Ct < 200E-9:
            i = 0
            j = 0
            for i in range(len(R_values)):
                for j in range(i, len(R_values)):
                    Rt = parallel(R_values[i], R_values[j])
                    if 2E3 < Rt < 150E3: 
                        f = 0.5/(Ct*(0.7*Rt + Rd*3))
                        if fsw*(1-tol) < f < fsw*(1+tol):
                            CT.append(Ct)
                            RT.append((R_values[i], R_values[j]))

    print(f'Se encontraron {len(CT)} opciones')
    for i in range(len(CT)):
        print('CT = ' + to_unit(CT[i]) + 'F con RT1 = ' + to_unit(RT[i][0]) + 'Ohm y RT2 = ' + to_unit(RT[i][1]) + 'Ohm, resultando en fsw = ' + to_unit(0.5/(CT[i]*(0.7*parallel(RT[i][0], RT[i][1]) + Rd*3)), 5) + 'Hz')

Se encontraron 0 opciones
Buscando con resistencias en paralelo

Se encontraron 5 opciones
CT = 1.5nF con RT1 = 6.8kOhm y RT2 = 100kOhm, resultando en fsw = 74.79kHz
CT = 2.2nF con RT1 = 4.7kOhm y RT2 = 56kOhm, resultando en fsw = 74.878kHz
CT = 2.2nF con RT1 = 6.8kOhm y RT2 = 12kOhm, resultando en fsw = 74.803kHz
CT = 4.7nF con RT1 = 2.2kOhm y RT2 = 27kOhm, resultando en fsw = 74.709kHz
CT = 4.7nF con RT1 = 2.7kOhm y RT2 = 8.2kOhm, resultando en fsw = 74.821kHz


Al final pusimos $C_T = 3.3nF$ y $R_T = 2.2k\Omega$ + 1 PRESET de $1k \Omega$ ajustable

$\Rightarrow f_{sw} = 75kHz$

## Divisor de tensión medida
$\overline{V_o} = 16V$

$V_{comp} = 2.55V$

$V_{ref} = 5.1V$

$V_{o_{comp}} = \frac{R_a}{R_a + R_b} \cdot \overline{V_o} \approx V_{comp}$

In [1]:
from utils import R_values, to_unit

Vcomp = 2.3212
Vo = 5.1

tol = 0.002
Ra = []
Rb = []

for ra in R_values:
	if 100 < ra < 150E3: 
		for rb in R_values:
			if 100 < rb < 150E3: 
				Vo_comp = Vo*ra/(ra + rb)
				if Vcomp*(1-tol) < Vo_comp < Vcomp*(1+tol):
					Ra.append(ra)
					Rb.append(rb)
        
print(f'Se encontraron {len(Ra)} opciones')
for i in range(len(Ra)):
    print('Ra = ' + to_unit(Ra[i]) + 'Ohm con Rb = ' + to_unit(Rb[i]) + 'Ohm, resultando en Vo_comp = ' + to_unit(Vo*Ra[i]/(Ra[i] + Rb[i]), 5) + 'V')

Se encontraron 6 opciones
Ra = 150Ohm con Rb = 180Ohm, resultando en Vo_comp = 2.3182V
Ra = 1kOhm con Rb = 1.2kOhm, resultando en Vo_comp = 2.3182V
Ra = 1.5kOhm con Rb = 1.8kOhm, resultando en Vo_comp = 2.3182V
Ra = 10kOhm con Rb = 12kOhm, resultando en Vo_comp = 2.3182V
Ra = 15kOhm con Rb = 18kOhm, resultando en Vo_comp = 2.3182V
Ra = 100kOhm con Rb = 120kOhm, resultando en Vo_comp = 2.3182V


Tomamos $R_a = 5.6k\Omega$ y $R_b = 33k\Omega$

$\Rightarrow V_{o_{comp}} = 2.3212$

Tomamos $R_a = 10k\Omega$ y $R_b = 12k\Omega$

$\Rightarrow V_{{ref}_{comp}} = 2.3182V$

## Compensación por realimentación

Como no tenemos la transferencia todavía no la hicimos